In [1]:
mode='baseline'
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
# from mne.decoding import CSP
from pyriemann.spatialfilters import CSP
from pyriemann.tangentspace import TangentSpace,FGDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.pipeline import make_pipeline,Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn import preprocessing
from sklearn.svm import SVC
from pyriemann.classification import MDM,FgMDM,MeanField
from pyriemann.tangentspace import TangentSpace,FGDA
import numpy as np
import moabb,sys,os
from moabb.datasets import BNCI2014001
from moabb.evaluations import CrossSessionEvaluation,WithinSessionEvaluation
from moabb.paradigms import FilterBankMotorImagery,FilterBankLeftRightImagery, LeftRightImagery,MotorImagery
from moabb.pipelines.utils import FilterBank 
from pyriemann.estimation import Covariances
import pyriemann
from san_utils import getfilters,all_metrics, label_map_fn #,bestfilters
# moabb.set_log_level("info")

In [6]:
!pip uninstall pyriemann

^C


In [ ]:
!pip install pyriemann

In [2]:
pipelines = {}
# plist = [1.00, 0.75, 0.50, 0.25, 0.10, 0.01, -0.01, -0.10, -0.25, -0.50, -0.75, -1.00]
# pipelines["FB_MeanField"] = make_pipeline(Covariances("oas"), MeanField(power_list=plist))
# pipelines["FB_FGDA_MeanField"] = make_pipeline(Covariances("oas"),FGDA(), MeanField(power_list=plist))
pipelines["FB_CSP_LDA"] = make_pipeline(Covariances("oas"),CSP(), LDA())
pipelines["FB_CSP_SVM"] = make_pipeline(Covariances("oas"),CSP(), SVC())
pipelines["FB_TS_LDA"] = make_pipeline(Covariances("oas"),TangentSpace(), LDA())
pipelines["FB_TS_SVM"] = make_pipeline(Covariances("oas"),TangentSpace(), SVC())
pipelines["FB_FGDA_TS_LDA"] = make_pipeline(Covariances("oas"),FGDA(),TangentSpace(), LDA())
pipelines["FB_FGDA_TS_SVM"] = make_pipeline(Covariances("oas"),FGDA(),TangentSpace(), SVC())
pipelines["FB_MDM"] = make_pipeline(Covariances("oas"), MDM(metric="riemann"))
pipelines["FB_FgMDM"] = make_pipeline(Covariances("oas"), FgMDM(metric="riemann"))

In [3]:
print(X_train.shape)
Xt=X_train;Xe=X_test

print(Xt.shape)
clf=Covariances("oas") #make_pipeline(Covariances("oas"),FGDA())
Xt=clf.fit_transform(Xt,y_train);Xe=clf.transform(Xe);
print(Xt.shape,y_train.shape)

clf=FGDA() #make_pipeline(Covariances("oas"),FGDA())
clf.fit(Xt,y_train);
Xt=clf.transform(Xt);Xe=clf.transform(Xe);
print(Xt.shape)

NameError: name 'X_train' is not defined

In [4]:
dataset = BNCI2014001()
dataset_name="BNCI2014001"
fmin = 8; fmax = 30; 
subjects = [1, 2, 3,4,5,6,7,8,9]; 
n_class=4; # value1='word_ass';value2='right_hand'
paradigm = MotorImagery(n_classes=n_class,fmin=fmin, fmax=fmax)
df = pd.DataFrame(columns=['dataset','session','channels','subject','n_filters','pipeline','acc',
                           'recall','precision','f1','kappa','sensitivity','specificity'])

flg=0
for subject in subjects:
    print("subject:",subject)
    X, y, meta = paradigm.get_data(dataset=dataset, subjects=[subject]) 
    nchan=X.shape[1]
    if flg==0:
        le,le_label_map=label_map_fn(y) 
        flg=1
    select_indices = list(np.where(meta["session"] == "session_T")[0])
    X0=X[select_indices,:];y0=y[select_indices]
    select_indices = list(np.where(meta["session"] == "session_E")[0])
    X1=X[select_indices,:];y1=y[select_indices]
#     ind0 = [i for i, value in enumerate(y0) if ((value == value1) or (value ==value2))]
#     ind1 = [i for i, value in enumerate(y1) if ((value == value1) or (value ==value2))]
#     X_train=X0[ind0];  X_test=X1[ind1]; y_train =y0[ind0] ; y_test=y1[ind1]
#     n_class=2
    y_train = le.transform(y0); y_test = le.transform(y1)
    X_train=X0;X_test=X1
    print(X_train.shape,np.unique(y_train))
    for pipeline in pipelines:
        print(pipeline)
        pipe=pipelines[pipeline]
        pipe.fit(X_train, y_train)
        y_pred=pipe.predict(X_test)
        acc,recall,precision,f1,kappa,sens,spec=all_metrics(y_test,y_pred)
        print(acc)
        df.loc[len(df)] = [dataset_name, "session_E", nchan, subject, 0, pipeline, acc,recall,precision,f1,kappa,sens,spec]

Choosing from all possible events


subject: 1
(288, 22, 1001) [0 1 2 3]
FB_CSP_LDA
0.6875
FB_CSP_SVM
0.6909722222222222
FB_TS_LDA


ValueError: expected square "a" matrix

In [11]:
df

,dataset,session,channels,subject,n_filters,pipeline,acc,recall,precision,f1,kappa,sensitivity,specificity


In [50]:
# saving the results
sfile_name = dataset_name+'/'+mode+'/Results_1band_baseline.csv'


if(os.path.isfile('./'+sfile_name) ==True):
    print("exists")
    df.to_csv(sfile_name, mode="a", index=False, header=False)
else:
    print("No.. so creating")
    df.to_csv(sfile_name,  index=False, header=True)    




No.. so creating
